# Modeling Stock Market Sentiment with CNNs and Keras

In this tutorial, we will build a CNN Network to predict the stock market sentiment based on a comment about the market.

## Setup

We will use the following libraries for our analysis:

* numpy - numerical computing library used to work with our data
* pandas - data analysis library used to read in our data from csv
* tensorflow - a lower level deep learning framework used for modeling
* keras - a higher level deep learning library that absracts away a lot of DL details. Keras will use Tensforflow in the background

We will also be using the python Counter object for counting our vocabulary items and we have a util module that extracts away a lot of the details of our data processing. Please read through the util.py to get a better understanding of how to preprocess the data for analysis.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import utils as utl
from collections import Counter

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.layers import Dense, Input, Flatten, Dropout, Merge
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

Using TensorFlow backend.


## Processing Data

We will train the model using messages tagged with SPY, the S&P 500 index fund, from [StockTwits.com](https://www.stocktwits.com). StockTwits is a social media network for traders and investors to share their views about the stock market. When a user posts a message, they tag the relevant stock ticker ($SPY in our case) and have the option to tag the messages with their sentiment – “bullish” if they believe the stock will go up and “bearish” if they believe the stock will go down.

Our dataset consists of approximately 100,000 messages posted in 2017 that are tagged with $SPY where the user indicated their sentiment. Before we get to our CNN Network we have to perform some processing on our data to get it ready for modeling.

#### Read and View Data

First we simply read in our data using pandas, pull out our message and sentiment data into numpy arrays. Let's also take a look at a few samples to get familiar with the data set.

In [2]:
# read data from csv file
data = pd.read_csv("data/StockTwits_SPY_Sentiment_2017.gz",
                   encoding="utf-8",
                   compression="gzip",
                   index_col=0)

# get messages and sentiment labels
messages = data.message.values
labels = data.sentiment.values

# View sample of messages with sentiment

for i in range(10):
    print("Messages: {}...".format(messages[i]),
          "Sentiment: {}".format(labels[i]))

Messages: $SPY crazy day so far!... Sentiment: bearish
Messages: $SPY Will make a new ATH this week. Watch it!... Sentiment: bullish
Messages: $SPY $DJIA white elephant in room is $AAPL. Up 14% since election. Strong headwinds w/Trump trade & Strong dollar. How many 7's do you see?... Sentiment: bearish
Messages: $SPY blocks above. We break above them We should push to double top... Sentiment: bullish
Messages: $SPY Nothing happening in the market today, guess I'll go to the store and spend some $.... Sentiment: bearish
Messages: $SPY What an easy call. Good jobs report: good economy, markets go up.  Bad jobs report: no more rate hikes, markets go up.  Win-win.... Sentiment: bullish
Messages: $SPY BS market.... Sentiment: bullish
Messages: $SPY this rally all the cheerleaders were screaming about this morning is pretty weak. I keep adding 2 my short at all spikes... Sentiment: bearish
Messages: $SPY Dollar ripping higher!... Sentiment: bearish
Messages: $SPY no reason to go down !... S

#### Check Message Lengths

We will also want to get a sense of the distribution of the length of our inputs. We check for the longest and average messages. We will need to make our input length uniform to feed the data into our model so later we will have some decisions to make about possibly truncating some of the longer messages if they are too long. We also notice that one message has no content remaining after we preprocessed the data, so we will remove this message from our data set.

In [3]:
messages_lens = Counter([len(x) for x in messages])
print("Zero-length messages: {}".format(messages_lens[0]))
print("Maximum message length: {}".format(max(messages_lens)))
print("Average message length: {}".format(np.mean([len(x) for x in messages])))

Zero-length messages: 0
Maximum message length: 635
Average message length: 75.64462136603174


In [4]:
messages, labels = utl.drop_empty_messages(messages, labels)

#### Preprocess Messages

Working with raw text data often requires preprocessing the text in some fashion to normalize for context. In our case we want to normalize for known unique "entities" that appear within messages that carry a similar contextual meaning when analyzing sentiment. This means we want to replace references to specific stock tickers, user names, url links or numbers with a special token identifying the "entity". Here we will also make everything lower case and remove punctuation.

In [5]:
messages = np.array([utl.preprocess_ST_message(message) for message in messages])

#### Generate Vocab to Index Mapping

We will use a Keras `Tokenizer` in order to generate our word index. The tockenizer takes our vocabulary and assigns each word a unique index from 1 to *VOCAB_SIZE*. Zero is reserved for padding which we will get to in a bit

In [6]:
VOCAB = sorted(list(set(messages)))
VOCAB_SIZE = len(VOCAB)

In [7]:
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(messages)

In [8]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 31975 unique tokens.


#### Encode Messages and Labels

We need to "translate" our text to number for our algorithm to take in as inputs. We call this translation an encoding. We encode our messages to sequences of numbers where each nummber is the word index from the mapping we made earlier. The phrase "I am bullish" would now look something like [1, 234, 5345] where each number is the index for the respective word in the message. We can do this very easily with our tokenizer by calling the `text_to_sequences` method. For our sentiment labels we will simply encode "bearish" as 0 and "bullish" as 1.

In [9]:
sequences = tokenizer.texts_to_sequences(messages)

In [10]:
labels = to_categorical(utl.encode_ST_labels(labels))

#### Pad Messages

The last thing we need to do is make our message inputs the same length. In our case, the average message length is 78 words so we will use a max length of around this amount. We need to Zero Pad the rest of the messages that are shorter. We will use a left padding that will pad all of the messages that are shorter than 244 words with 0s at the beginning. So our encoded "I am bullish" messages goes from [1, 234, 5345] (length 3) to [0, 0, 0, 0, 0, 0, ... , 0, 0, 1, 234, 5345] (length 80). Keras has a build in processing function called `pad_sequences` to do this for us.

In [11]:
MAX_SEQUENCE_LENGTH = 80
cnn_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

#### Train and Validation Split

The last thing we do is split our data into tranining and validation sets. Typically we will want a test set as well but we will skip this for demonstration purposes.

In [12]:
VALIDATION_SPLIT = .2
num_validation_samples = int(VALIDATION_SPLIT * cnn_data.shape[0])

In [13]:
indices = np.arange(cnn_data.shape[0])
np.random.shuffle(indices)
cnn_data = cnn_data[indices]
labels = labels[indices]

In [14]:
x_train = cnn_data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = cnn_data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

## Building and Training our CNN Network

In this section we will load our pretrained word embeddings and build out CNN Model.

#### Glove Embeddings

For this example we will use the Twitter GloVe embeddings that can be found here https://nlp.stanford.edu/projects/glove/. We have our embeddings saved in a text file in our data directory so first we load parse these and load them in to a dictionary.

Next we get the mean and standard devation of all embedding values. The pretrained GloVe embeddings won't contain all of the words in our vocabularly so we will seed our embedding matrix for our vocabularly with random draws from a normal distribution with mean *emb_mean* and standard deviation *emb_std*. Next we iterate through all of the words in our vocabulary and set our word embeddings to the GloVe vectors where they are available.

In [26]:
EMBEDDING_DIM = 50
EMBEDDING_FILE = 'data/glove.twitter.27B.50d.txt'

def get_embed_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')

In [18]:
embeddings_index = dict(get_embed_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [19]:
embeddings_values=list(embeddings_index.values())
all_embs = np.stack(embeddings_values)
emb_mean,emb_std = all_embs.mean(), all_embs.std()

In [20]:
embedding_matrix = np.random.normal(emb_mean, emb_std, (len(word_index) + 1, EMBEDDING_DIM))

In [21]:
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

In [22]:
embedding_matrix.shape

(31976, 50)

#### Model and Training

Here we build our Convoluational Neural Network using Keras. We will use the architecture from the Yoon Kim model (https://arxiv.org/abs/1408.5882) with some adjustments. We first start with our embeddings layer and then have 3 parallel 1D convulational layers with 128 filters and sizes [3, 4, 5] respectively. We concatenate these results, pass to a dropout layer for regularization, and then to a fully connected layer with relu activation and finally a softmax layer to make out predictions.

Here we also define that we will use a categorical crossentropy loss funcation and an Adam optimizer.

In [27]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index, trainable=False, extra_conv=True):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    l_merge = Merge(mode='concat', concat_axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    #x = Dropout(0.5)(x)

    preds = Dense(labels_index, activation='softmax')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])

    return model

In [24]:
model = ConvNet(embedding_matrix, MAX_SEQUENCE_LENGTH, len(word_index)+1, EMBEDDING_DIM, 
                len(labels[0]), False)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


and now we train!

In [25]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=50, batch_size=128)

Train on 77574 samples, validate on 19393 samples
Epoch 1/50
77574/77574 [==============================] - 12s - loss: 0.6753 - acc: 0.5859 - val_loss: 0.6544 - val_acc: 0.6047
Epoch 2/50
77574/77574 [==============================] - 9s - loss: 0.6345 - acc: 0.6348 - val_loss: 0.6146 - val_acc: 0.6532
Epoch 3/50
77574/77574 [==============================] - 9s - loss: 0.6028 - acc: 0.6689 - val_loss: 0.5894 - val_acc: 0.6806
Epoch 4/50
77574/77574 [==============================] - 9s - loss: 0.5756 - acc: 0.6942 - val_loss: 0.5797 - val_acc: 0.6917
Epoch 5/50
77574/77574 [==============================] - 9s - loss: 0.5539 - acc: 0.7101 - val_loss: 0.5636 - val_acc: 0.6999
Epoch 6/50
77574/77574 [==============================] - 9s - loss: 0.5345 - acc: 0.7219 - val_loss: 0.5570 - val_acc: 0.7073
Epoch 7/50
77574/77574 [==============================] - 9s - loss: 0.5184 - acc: 0.7345 - val_loss: 0.5548 - val_acc: 0.7085
Epoch 8/50
77574/77574 [==============================] - 9s